In [ ]:
# ========================
# Install Required Libraries
# ========================
!pip install opencv-python-headless
!pip install tensorflow
!pip install matplotlib seaborn scikit-learn

# ========================
# Import All Libraries
# ========================
import os
import random
import shutil
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

print("✅ Libraries installed and imported successfully.")


✅ Libraries installed and imported successfully.


In [ ]:
# ========================
# Download UCF101 Dataset
# ========================
!wget --no-check-certificate https://crcv.ucf.edu/data/UCF101/UCF101.rar

# ========================
# Install Unrar Tool
# ========================
!apt install unrar -y

# ========================
# Extract UCF101 Dataset
# ========================
!unrar x UCF101.rar

# After this step, videos will be extracted into /content/UCF-101/
print("✅ Dataset downloaded and extracted.")


Streaming output truncated to the last 5000 lines.
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g06_c06.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g06_c07.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c01.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c02.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c03.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c04.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c05.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c06.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g07_c07.avi             62%  OK 
Extracting  UCF-101/PlayingGuitar/v_PlayingGuitar_g08_c01.avi             62%  OK 
Extracting  UCF-101/PlayingGu

In [ ]:
# ========================
# Choose 2 classes: JumpingJack and PushUps
# ========================
selected_classes = ["JumpingJack", "PushUps"]

# ========================
# Create a directory for organized dataset
# ========================
os.makedirs('dataset', exist_ok=True)
for cls in selected_classes:
    os.makedirs(f'dataset/{cls}', exist_ok=True)

# ========================
# Function to extract frames from a video
# ========================
def extract_frames(video_path, save_dir, img_size=64, frame_count=16):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (img_size, img_size))
        frames.append(frame)
    cap.release()

    # Normalize frames
    frames = np.array(frames) / 255.0

    # Keep only frame_count frames
    if len(frames) > frame_count:
        frames = frames[:frame_count]
    else:
        while len(frames) < frame_count:
            frames = np.append(frames, [frames[-1]], axis=0)

    # Save frames
    os.makedirs(save_dir, exist_ok=True)
    for idx, frame in enumerate(frames):
        frame = (frame * 255).astype(np.uint8)
        cv2.imwrite(os.path.join(save_dir, f'frame_{idx:03d}.png'), frame)

# ========================
# Select 5 videos per class and preprocess
# ========================
for cls in selected_classes:
    src_folder = f'UCF-101/{cls}'
    videos = os.listdir(src_folder)
    selected_videos = random.sample(videos, 5)  # Pick 5 random videos

    for idx, video in enumerate(selected_videos):
        video_path = os.path.join(src_folder, video)
        save_folder = f'dataset/{cls}/clip_{idx+1}'
        extract_frames(video_path, save_folder, img_size=64, frame_count=16)

print("✅ Videos selected and frames extracted into 'dataset/' folder.")


✅ Videos selected and frames extracted into 'dataset/' folder.


In [ ]:
# ========================
# Load Frames into Numpy Arrays
# ========================
def load_dataset(data_dir, img_size=64, frame_count=16):
    X = []
    y = []

    for cls in os.listdir(data_dir):
        cls_path = os.path.join(data_dir, cls)
        for clip_folder in os.listdir(cls_path):
            clip_path = os.path.join(cls_path, clip_folder)
            frames = []
            for frame_file in sorted(os.listdir(clip_path)):
                frame_path = os.path.join(clip_path, frame_file)
                img = cv2.imread(frame_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = img / 255.0  # Normalize again just to be sure
                frames.append(img)
            X.append(np.array(frames))
            y.append(cls)

    return np.array(X), np.array(y)

# Load
X, y = load_dataset('dataset', img_size=64, frame_count=16)
print(f"✅ Loaded dataset with shape: {X.shape}, Labels shape: {y.shape}")


✅ Loaded dataset with shape: (10, 16, 64, 64, 3), Labels shape: (10,)


In [ ]:
# Convert string labels to integers
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"✅ Encoded labels: {y_encoded}")
print(f"Classes: {le.classes_}")


✅ Encoded labels: [1 1 1 1 1 0 0 0 0 0]
Classes: ['JumpingJack' 'PushUps']


In [ ]:
# Generator Model
def build_generator(latent_dim=100, frames=16, img_size=64, channels=3):
    model = tf.keras.Sequential([
        layers.Dense(8*8*128, use_bias=False, input_shape=(latent_dim,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Reshape((1,8,8,128)),  # Shape: (1 frame, 8, 8, 128)

        layers.Conv3DTranspose(128, (2,4,4), strides=(4,2,2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv3DTranspose(64, (4,4,4), strides=(2,2,2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        layers.Conv3DTranspose(channels, (4,4,4), strides=(2,2,2), padding='same', activation='tanh')
    ])
    return model


In [ ]:
# Discriminator Model
def build_discriminator(frames=16, img_size=64, channels=3):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=(frames, img_size, img_size, channels)),

        layers.Conv3D(64, (4,4,4), strides=(2,2,2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Conv3D(128, (4,4,4), strides=(2,2,2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1)
    ])
    return model


In [ ]:
# Initialize
latent_dim = 100
generator = build_generator(latent_dim=latent_dim)
discriminator = build_discriminator()

# Optimizers
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
disc_optimizer = tf.keras.optimizers.Adam(1e-4)

# Loss function
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# Show model summary
generator.summary()
discriminator.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 8192)           │       819,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 8192)           │        32,768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 1, 8, 8, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose                │ (None, 4, 16, 16, 128) │       524,288 │
│ (Conv3DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 4, 16, 16, 128) │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 4, 16, 16, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_1              │ (None, 8, 32, 32, 64)  │       524,288 │
│ (Conv3DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 32, 32, 64)  │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_2 (LeakyReLU)       │ (None, 8, 32, 32, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_2              │ (None, 16, 64, 64, 3)  │        12,291 │
│ (Conv3DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,913,603 (7.30 MB)

 Trainable params: 1,896,835 (7.24 MB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 8, 32, 32, 64)  │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 8, 32, 32, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 8, 32, 32, 64)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_1 (Conv3D)               │ (None, 4, 16, 16, 128) │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 4, 16, 16, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4, 16, 16, 128) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │       131,073 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 667,841 (2.55 MB)

 Trainable params: 667,841 (2.55 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
batch_size = 2

# Create TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices(X).shuffle(100).batch(batch_size)


In [ ]:
# Training step function
@tf.function
def train_step(real_videos):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_videos = generator(noise, training=True)

        real_output = discriminator(real_videos, training=True)
        fake_output = discriminator(generated_videos, training=True)

        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + cross_entropy(tf.zeros_like(fake_output), fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss



In [ ]:
# Full training loop
def train(dataset, epochs):
    gen_losses = []
    disc_losses = []

    for epoch in range(epochs):
        for real_videos in dataset:
            gen_loss, disc_loss = train_step(real_videos)

        gen_losses.append(gen_loss.numpy())
        disc_losses.append(disc_loss.numpy())

        print(f'Epoch {epoch+1}, Generator Loss: {gen_loss.numpy():.4f}, Discriminator Loss: {disc_loss.numpy():.4f}')

    return gen_losses, disc_losses

# Start training
gen_losses, disc_losses = train(train_dataset, epochs=50)


Epoch 1, Generator Loss: 0.7490, Discriminator Loss: 0.9126
Epoch 2, Generator Loss: 0.9661, Discriminator Loss: 0.7088
Epoch 3, Generator Loss: 1.2676, Discriminator Loss: 0.3361
Epoch 4, Generator Loss: 1.8861, Discriminator Loss: 0.5870
Epoch 5, Generator Loss: 1.1574, Discriminator Loss: 0.4454
Epoch 6, Generator Loss: 1.9523, Discriminator Loss: 0.2189
Epoch 7, Generator Loss: 2.0364, Discriminator Loss: 0.2499
Epoch 8, Generator Loss: 2.4139, Discriminator Loss: 0.1150
Epoch 9, Generator Loss: 2.6013, Discriminator Loss: 0.1381
Epoch 10, Generator Loss: 2.9373, Discriminator Loss: 0.1163
Epoch 11, Generator Loss: 2.2734, Discriminator Loss: 0.1849
Epoch 12, Generator Loss: 3.8699, Discriminator Loss: 0.3145
Epoch 13, Generator Loss: 3.4424, Discriminator Loss: 0.0348
Epoch 14, Generator Loss: 3.2644, Discriminator Loss: 0.0832
Epoch 15, Generator Loss: 3.6847, Discriminator Loss: 0.1787
Epoch 16, Generator Loss: 2.9132, Discriminator Loss: 0.0650
Epoch 17, Generator Loss: 4.4076,

In [ ]:
# Save as AVI video
import cv2
import numpy as np
os.makedirs('generated_videos', exist_ok=True)

video_path = 'generated_videos/fake_video.avi'

# Generate video frames using the trained generator
num_frames = 16  # Number of frames to generate
latent_dim = 100 # Latent dimension of the generator
noise = tf.random.normal([1, latent_dim])  # Generate noise for the generator
generated_video = generator(noise, training=False)  # Generate video frames
generated_video = generated_video[0] # Get the video from the batch output
generated_video = generated_video.numpy() # Convert to numpy array
generated_video = (generated_video * 127.5 + 127.5).astype(np.uint8) # Rescale to 0-255

# Define video writer
height, width, _ = generated_video[0].shape
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'DIVX'), 8, (width, height))

# Write each frame
for frame in generated_video:
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert RGB -> BGR
    out.write(frame_bgr)

out.release()
print(f"✅ Generated fake video saved at: {video_path}")


✅ Generated fake video saved at: generated_videos/fake_video.avi
